In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

/home/user_3/medistream-recsys/Script


# product, orders, users, product_category 합치기

In [ ]:
# data load
df_orders = pd.read_json('/fastcampus-data/orders/orders.json')
df_product = pd.read_json('/fastcampus-data/products/products.json')
df_user = pd.read_json('/fastcampus-data/users/users.json', lines=True)
df_product_category = pd.read_json('/fastcampus-data/products/productCategories.json/products')

# orders 전처리
columns = ['_id','date_created','date_placed','date_updated','date_closed','date_paid',\
 'customer_id','status_id','payment_method_id','shipping_method_id']
df_orders = key_to_element(columns,df_orders)
df_orders = dict_to_column(['items'],df_orders)
df_orders = key_to_element(['id','variant_id','product_id'],df_orders)
df_orders = df_orders.rename(columns={'product_id':'product_ids'})

# product 전처리
df_product = key_to_element(['_id','date_created','date_updated'],df_product)
df_product = df_product.rename(columns={'_id':'product_ids'})

# user 전처리
df_user = df_user.drop_duplicates(['realname_sha_id'], keep='last')
df_user = df_user[~df_user['customer_id'].isna()]

# product_category 전처리
df_product_category = df_product_category.rename(columns={'_id':'product_ids'})
df_product_category = key_to_element(['product_ids'],df_product_category)

In [81]:
# 모든 파일 merge
df_order_product = pd.merge(df_orders,df_product, how='left', on='product_ids')
df_order_product_user = pd.merge(df_order_product,df_user, how='left', on='customer_id')
df_all_sum = pd.merge(df_order_product_user,df_product_category, how='left',on='product_ids')

In [116]:
# orders 있지만 products 없는 ids 확인
order_product_id_set= set(df_orders.product_ids)
meta_product_id_set= set(df_product.product_ids)
len(order_product_id_set), len(meta_product_id_set)
delete_product_id = order_product_id_set - meta_product_id_set
delete_product_id

{'5e957f03bfe42609447858e5',
 '5f1009e2221c8b5f77c8e54f',
 '5f1009e3221c8b5f77c8e550',
 '5f1009e4221c8b5f77c8e551',
 '5f55d798a0b91001bf48f55e',
 '5f892894512031633431e6c5',
 '5f927537512031633431f672',
 '6064035f0ccc56001bc7a2fd',
 None}

In [102]:
# orders 있지만 products 없는 ids 드롭
df_all_sum = df_all_sum[~df_all_sum['product_ids'].isna()]
df_all_sum = df_all_sum[~df_all_sum['product_ids'].isin(delete_product_id)]

In [133]:
# 저장한 파일 불러오기
df = pd.read_json('/fastcampus-data/all_sum.json')